<a href="https://colab.research.google.com/github/Piyushkumar1307/Text-Summarizer/blob/main/Text-Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [2]:
pip install mtranslate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for mtranslate: filename=mtranslate-1.8-py3-none-any.whl size=3697 sha256=08b8970a76e1d3c61cb41ddc3c4db79048d6ffa67aaef6994a65fc3c2e203a5d
  Stored in directory: /root/.cache/pip/wheels/a2/20/13/93c62f314d4a29db25b1b24b2c38f79eb1beb558c727b894a3
Successfully built mtranslate


In [3]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.3 MB/s eta 0:00:00


In [4]:
!pip install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=adae6d33e13b9f8d065439ecf22de2c7620d977f537812a8875f58e730a205d8
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [6]:
import mtranslate
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from heapq import nlargest
import time
from spellchecker import SpellChecker
from rouge import Rouge
import nltk
import langdetect

# Dictionary of Indian languages and their ISO codes
indian_languages = {
    "Bengali": "bn",
    "Gujarati": "gu",
    "Hindi": "hi",
    "Kannada": "kn",
    "Malayalam": "ml",
    "Marathi": "mr",
    "Nepali": "ne",
    "Oriya": "or",
    "Punjabi": "pa",
    "Tamil": "ta",
    "Telugu": "te",
    "Urdu": "ur"
}

def summarize(text, percentage):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    freq_table = {}
    for word in words:
        if word not in stop_words:
            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

    # Perform NER and exclude recognized entities from frequency table
    entities = nltk.ne_chunk(nltk.pos_tag(words))
    for entity in entities:
        if isinstance(entity, nltk.tree.Tree):
            entity_words = [word for word, tag in entity.leaves()]
            if len(entity_words) > 1:
                entity_text = ' '.join(entity_words)
                for word in entity_words:
                    del freq_table[word]
                freq_table[entity_text] = words.count(entity_text)

    sentences = sent_tokenize(text)
    sentence_scores = {}
    for sent in sentences:
        for word in word_tokenize(sent.lower()):
            if word in freq_table:
                if sent in sentence_scores:
                    sentence_scores[sent] += freq_table[word]
                else:
                    sentence_scores[sent] = freq_table[word]

    n = int(len(sentences) * percentage)
    summary_sentences = nlargest(n, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    return summary


def translate_text(text, target_language):
    translation = mtranslate.translate(text, target_language)
    return translation


def main(target_language=None):
    text = input("Enter text to summarize and translate: ")
    print("Length of original text:", len(text))

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if not word in stop_words]
    filtered_text = ' '.join(filtered_words)

    # Perform spell checking and exclude recognized entities
    spell = SpellChecker()
    misspelled = spell.unknown(filtered_words)
    entities = nltk.ne_chunk(nltk.pos_tag(words))
    recognized_entities = []
    for entity in entities:
        if isinstance(entity, nltk.tree.Tree):
            entity_words = [word for word, tag in entity.leaves()]
            if len(entity_words) > 1:
                entity_text = ' '.join(entity_words)
                recognized_entities.append(entity_text)
    corrections = {}
    for word in filtered_words:
        if word in recognized_entities:
            corrections[word] = word
        elif word in misspelled:
            if word == "'s":
                corrections[word] = word
            else:
                correction = spell.correction(word)
                corrections[word] = correction
        else:
            corrections[word] = word
    corrected_words = [corrections[word] for word in filtered_words]
    corrected_text = ' '.join(corrected_words)
    print("Corrected text:")
    print(corrected_text)
    print("Incorrect words and their corrections:")
    for word in corrections:
        if corrections[word] != word:
            print(word, "->", corrections[word])

    percentage = float(input("Enter the percentage of summarization (e.g. 0.2 for 20%): "))

    start_time = time.time()
    summary = summarize(corrected_text, percentage)
    end_time = time.time()

    print("Summary:\n", summary)
    print("Length of summary:", len(summary))
    print("Time taken:", end_time - start_time, "seconds")

    if target_language is None:
    # Ask user if they want to translate the summary
       translate_option = input("Do you want to translate the summary? (y/n): ")
       if translate_option.lower() == 'y':
          print("List of Indian languages:")
          for lang in indian_languages:
              print(lang)
          target_language_name = input("Enter target language name: ")
          target_language = indian_languages.get(target_language_name)
          if not target_language:
            print("Invalid language name.")
            return
       else:
        print("Summary will not be translated.")

    else:
        target_language_name = [k for k, v in indian_languages.items() if v == target_language][0]
        print("Target language: ", target_language_name)

    translated_summary = translate_text(summary, target_language)
    print("Translated summary:\n", translated_summary)

# Calculate ROUGE score
    reference_summary = input("Enter the reference summary: ")
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference_summary)
    print("Accuracy: ", scores[0]['rouge-1']['f'])

if __name__ == '__main__':
    main()


Enter text to summarize and translate: Clint Barton's daghter vanishes and the rest of his family disintegrates due to Thanos' snapping his fingers. Nebula and Tony Stark are stranded in space, but Carol Danvers returns them to Earth. Thor beheads Thanos to avoid using the Stones for further nefarious purposes. Five years later, Scott Lang escapes from the quantum realm and discovers that his daughter Cassie is now a teenager and that Hope van Dyne has disappeared. Rogers is leading grief counseling sessions for survivors, while Romanoff is keeping watch over Earth and the rest of the univese.
Length of original text: 561
Corrected text:
Clint Barton 's daughter vanishes rest family disintegrates due Thanos ' snapping fingers . Nebula Tony Stark stranded space , Carol Danvers returns Earth . Thor beheads Thanos avoid using Stones nefarious purposes . Five years later , Scott Lang escapes quantum realm discovers daughter Cassie teenager Hope van Dyne disappeared . Rogers leading grief c